In [1]:
import gym
import numpy as np
import FerchNN
from random import sample

In [4]:
env = gym.make("MountainCarContinuous-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
env.action_space.high

array([ 1.], dtype=float32)

In [6]:
mean_nn = FerchNN.NN_Network(input_size = env.observation_space.shape[0], lr =0.000001, output_size = env.action_space.shape[0] ,  activation = "linear")
std_nn = FerchNN.NN_Network(input_size = env.observation_space.shape[0], lr =0.000001, output_size = env.action_space.shape[0] , activation = "linear")

In [7]:
def choose_action(state):
    
    action = np.zeros(env.action_space.shape[0])
    mean = mean_nn.predict(state)
    std = std_nn.predict(state)
    #std_aux= np.log(1+np.power(np.e, std))+0.00001
    std_aux = np.power(std,2)+0.00001
    #std_aux = np.zeros(env.action_space.shape)
    
    for i in range(env.action_space.shape[0]):
        
        action[i] = np.random.normal(mean[i], std_aux[i])
        
        if action[i]>= env.action_space.high[i]:
            action[i] = env.action_space.high[i]
        
        if action[i]<= env.action_space.low[i]:
            action[i] = env.action_space.low[i]
            
        
    
    #std = std_aux
    #print(mean, std_aux)
    return action, mean, std 

In [8]:
def compute_gradient(action, reward, mean, std):
    
    #std_aux= np.log(1+np.power(np.e, std))+0.00001
    std_aux = np.power(std,2)+0.00001
    mean_gradient = -reward*((action - mean)/np.power(std_aux,2))
    #mean_gradient = (mean_gradient)/np.linalg.norm(mean_gradient)
     
    #std_gradient = -reward*((np.power((action-mean), 2)-np.power(std_aux,2))/np.power(std_aux,3))*( 1/ (1+np.power(np.e,-std)))
    std_gradient = -reward*((np.power((action-mean), 2)-np.power(std_aux,2))/np.power(std_aux,3))*2*std
    #std_gradient = (std_gradient)/ np.linalg.norm(std_gradient)
    
    
    return mean_gradient, std_gradient

In [9]:
def compute_cumulative_rewards(rewards, gamma = 0.99):
    
    r_aux = np.zeros(len(rewards))
    
    for i in range(len(rewards)):
        discounted_rewards = 0
        
        for j in range(i, len(rewards)): 
            discounted_rewards = discounted_rewards + np.power(gamma, j-i)*rewards[j]
        
        r_aux [i] = discounted_rewards
    
    return r_aux

In [10]:
def reinforce(actions, means, stds, rewards):
    
    r = compute_cumulative_rewards(rewards)
    
    assert len(actions)== len(means)
    assert len(means) == len(stds)
    assert len(stds) == len(rewards)
    
    for i in range(len(actions)):
        mean_gradient, std_gradient  = compute_gradient(actions[i], r[i], means[i], stds[i])
        mean_nn.fit(None,None, dloss = mean_gradient)
        std_nn.fit(None, None, dloss = std_gradient)

In [ ]:
n_episodes = 100000
horizon = 1000
for i_episode in range(n_episodes):
    state = env.reset()
    
    actions = []
    means = []
    stds = []
    rewards = []
    time = 0
    for t in range(horizon):
        time+=1
        env.render()
        ########################################
        action, mean, std = choose_action(state)
        actions.append(action)
        means.append(mean)
        stds.append(std)
        ########################################
        next_state, reward, done, info = env.step(action)
        rewards.append(reward)
        
        state = next_state
        if done:
            break
    reinforce(actions, means, stds, rewards)
    
    print("-------------------------------")
    print("Episode finished after {} timesteps".format(time+1))
    print(i_episode, " Reward ", np.array(rewards).sum())
    print(i_episode, " Mean ", np.array(means).mean())
    print(i_episode, " Std ", np.log(1+np.power(np.e, np.array(stds))).mean())
    
env.close()

D:\Fernando\Documents\TCC\continuous_action_space\FerchNN.py:210: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if dloss != None:


-------------------------------
Episode finished after 728 timesteps
0  Reward  27.3972164576
0  Mean  29.6388289482
0  Std  29.7360229463
-------------------------------
Episode finished after 1000 timesteps
1  Reward  -12.7709888863
1  Mean  -0.355146836309
1  Std  0.571770687246
-------------------------------
Episode finished after 1000 timesteps
2  Reward  -13.0392206659
2  Mean  -0.356287963795
2  Std  0.571943455752
-------------------------------
Episode finished after 1000 timesteps
3  Reward  -12.9547970635
3  Mean  -0.35658995421
3  Std  0.571900060076
-------------------------------
Episode finished after 1000 timesteps
4  Reward  -13.4299151025
4  Mean  -0.357535268418
4  Std  0.572262748734
-------------------------------
Episode finished after 1000 timesteps
5  Reward  -13.2166180743
5  Mean  -0.35658258891
5  Std  0.572391263823
-------------------------------
Episode finished after 1000 timesteps
6  Reward  -13.0510593864
6  Mean  -0.356162623723
6  Std  0.572318332308